In [12]:
come_follow_me_base_link = "https://www.churchofjesuschrist.org/study/come-follow-me?lang=eng"
base_link = "https://www.churchofjesuschrist.org"

In [42]:
import aiometer
import asyncio
import json
import httpx
from tqdm import tqdm

In [4]:
from bs4 import BeautifulSoup

In [46]:
output_file = "come_follow_me.jsonl"

with open(output_file, "w") as f:
    with httpx.Client() as client:
        response = client.get(come_follow_me_base_link)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        # Print top-level links
        links = [link_element['href'] for link_element in soup.find_all("a")[2:]]
        content_links = []
        for link in links:
            link = base_link + link
            response = client.get(link)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")
            content_links = content_links + [link_element['href'] for link_element in soup.find_all("a")[3:]]

        for content_link in tqdm(content_links):
            link = base_link + content_link
            response = client.get(link)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")
            # Find div with content id
            # content_div = soup.find("div", {"id": "content"})
            # Find div with manifest id
            manifest_div = soup.find("nav", {"class": "manifest"})
            # Find all links
            try:
                links = [link_element['href'] for link_element in manifest_div.find_all("a")][1:]
                for link in tqdm(links):
                    link = base_link + link
                    response = client.get(link)
                    response.raise_for_status()
                    soup = BeautifulSoup(response.text, "html.parser")
                    # Find div with content id
                    content_div = soup.find("div", {"class": "body-block"})
                    text = content_div.get_text()
                    jsonl = {"text": text}
                    f.write(json.dumps(jsonl) + "\n")
            except AttributeError:
                pass

 76%|███████▋  | 26/34 [10:14<03:09, 23.65s/it]


ReadTimeout: The read operation timed out

In [47]:
# Open the file to grab all the saved text
with open(output_file, "r") as f:
    lines = f.readlines()
    print(len(lines))

1627


In [50]:
# Randomly split 5% of the data into a validation set
import random
random.seed(42)
random.shuffle(lines)
num_validation_samples = int(len(lines) * 0.05)
num_train_samples = len(lines) - num_validation_samples
train_lines = lines[:num_train_samples]
validation_lines = lines[num_train_samples:]

In [51]:
print(len(train_lines))
print(len(validation_lines))
print(len(train_lines) + len(validation_lines))

1546
81
1627


In [ ]:
train_lines = [json.loads(line) for line in train_lines]
validation_lines = [json.loads(line) for line in validation_lines]

In [61]:
training_file = "model/data/train.json"
with open(training_file, "r") as f:
    data = json.load(f)
    new_data = data + train_lines
with open(training_file, "w") as f:
    json.dump(new_data, f)

validation_file = "model/data/validation.json"
with open(validation_file, "r") as f:
    data = json.load(f)
    new_data = data + validation_lines
with open(validation_file, "w") as f:
    json.dump(new_data, f)